In [1]:
import pandas as pd

In [5]:
path = 'C:/Users/zolboy/Documents/GitHub/Machine_Learning_OpenClassrooms/Customer Segmentation/data/'
olist_customers_dataset = pd.read_csv(path + 'olist_customers_dataset.csv', sep=',')
olist_geolocation_dataset = pd.read_csv(path + 'olist_geolocation_dataset.csv', sep=',')
olist_order_items_dataset = pd.read_csv(path + 'olist_order_items_dataset.csv', sep=',')
olist_order_payments_dataset = pd.read_csv(path + 'olist_order_payments_dataset.csv', sep=',')
olist_order_reviews_dataset = pd.read_csv(path + 'olist_order_reviews_dataset.csv', sep=',')
olist_orders_dataset = pd.read_csv(path + 'olist_orders_dataset.csv', sep=',')
olist_products_dataset = pd.read_csv(path + 'olist_products_dataset.csv', sep=',')
olist_sellers_dataset = pd.read_csv(path + 'olist_sellers_dataset.csv', sep=',')
product_category_name_translation = pd.read_csv(path + 'product_category_name_translation.csv', sep=',')


All_Datasets = {'olist_customers_dataset' : olist_customers_dataset, 'olist_geolocation_dataset' : olist_geolocation_dataset, 'olist_order_items_dataset' : olist_order_items_dataset, 'olist_order_payments_dataset' : olist_order_payments_dataset, 'olist_order_reviews_dataset' : olist_order_reviews_dataset, 'olist_orders_dataset' : olist_orders_dataset, 'olist_products_dataset' : olist_products_dataset, 'olist_sellers_dataset' : olist_sellers_dataset, 'product_category_name_translation' : product_category_name_translation}


In [13]:
# Befor merge the data we need to see our connection between files csv (databases connection)
for key,value in All_Datasets.items():
    value.info()
    print('------------')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             10001

In [ ]:
# Now we need to merge our data